In [8]:
import pandas as pd
import numpy as np
import warnings
from scipy.optimize import linprog as lp

# -------------------------
# Solver function (as in the original code)
# -------------------------
def solve_subsistence_problem(FoodNutrients, Prices, dietmin, dietmax, max_weight=None, tol=1e-6):
    """
    Solve the Subsistence Cost Problem.
    
    FoodNutrients: pd.DataFrame with rows = nutrients, columns = foods (values per 100 g)
    Prices: pd.Series of prices (per 100 g) for foods.
    dietmin: pd.Series of minimum daily intakes (in same units as FoodNutrients).
    dietmax: pd.Series of maximum daily intakes.
    """
    try: 
        p = Prices.apply(lambda x: x)
    except AttributeError:
        warnings.warn("Prices have no units. BE CAREFUL! We're assuming prices are per hectogram!")
        p = Prices

    p = p.dropna()

    # Use only those foods for which we have both price and nutrition info.
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]
    
    # Keep only nutritional information for those foods and fill missing nutrient values with 0.
    Aall = FoodNutrients[p.index].fillna(0)
    
    # Separate the constraints into "min" and "max"
    Amin = Aall.loc[Aall.index.intersection(dietmin.index)]
    Amin = Amin.reindex(dietmin.index, axis=0)
    
    Amax = Aall.loc[Aall.index.intersection(dietmax.index)]
    Amax = Amax.reindex(dietmax.index, axis=0)
    
    # For minimum constraints, multiply by -1 so that they become <= constraints.
    A = pd.concat([Amin, -Amax])
    b = pd.concat([dietmin, -dietmax])  # note sign change for max constraints
    
    # Ensure the ordering of p, A, b is consistent.
    A = A.reindex(p.index, axis=1)
    A = A.reindex(b.index, axis=0)
    
    # Optionally, add a constraint on the total diet weight.
    if max_weight is not None:
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight

    # Solve the LP: minimize cost pᵀx subject to A x ≤ b.
    result = lp(p, -A, -b, method='highs')
    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x, index=p.index)
    else:
        warnings.warn(result.message)
        result.diet = pd.Series(result.x, index=p.index) * np.nan  
    
    return result

# -------------------------
# Process the foods CSV
# -------------------------
# Read the foods file.
foods_df = pd.read_csv("NEWTJBase4.csv")

# Compute cost per 100 g.
# price is in dollars, package_size in grams.
foods_df['cost_per_100g'] = (foods_df['price'] / foods_df['package_size']) * 100

# Convert nutrient values from “per serving” to “per 100 g”.
# (Assumes serving_size is given in grams.)
foods_df['Energy'] = (foods_df['calories_per_serving'] / foods_df['serving_size']) * 100
foods_df['Protein'] = (foods_df['Protein'] / foods_df['serving_size']) * 100
foods_df['Dietary Fiber'] = (foods_df['Dietary Fiber'] / foods_df['serving_size']) * 100
foods_df['Total Carbohydrate'] = (foods_df['Total Carbohydrate'] / foods_df['serving_size']) * 100
foods_df['Calcium'] = (foods_df['Calcium'] / foods_df['serving_size']) * 100
foods_df['Iron'] = (foods_df['Iron'] / foods_df['serving_size']) * 100
foods_df['Sodium'] = (foods_df['Sodium'] / foods_df['serving_size']) * 100

# Aggregate duplicate food names by grouping.
# For prices, we take the minimum cost; for nutrients, we use the mean.
Prices = foods_df.groupby('name', sort=False)['cost_per_100g'].min()
nutrient_cols = ['Energy', 'Protein', 'Dietary Fiber', 'Total Carbohydrate', 'Calcium', 'Iron', 'Sodium']
FoodNutrients = foods_df.groupby('name', sort=False)[nutrient_cols].mean().T

# -------------------------
# Process the minimum DRIs CSV
# -------------------------
# Read the DRIs minimum file.
dri_min_df = pd.read_csv("cleanedmins.csv")
min_nutrients = ["Energy", "Protein", "Fiber, total dietary", "Calcium, Ca", 
                 "Carbohydrate, by difference", "Iron, Fe"]
dri_min_df = dri_min_df[dri_min_df['Nutrition'].isin(min_nutrients)].copy()
dri_min_df.set_index('Nutrition', inplace=True)

# Choose an age/sex group (e.g., "M 19-30").
group = "M 19-30"
diet_min = dri_min_df[group].copy()

# Adjust units for nutrients that are in mg (Calcium and Iron).
diet_min.loc["Calcium, Ca"] = diet_min.loc["Calcium, Ca"] / 1000.0
diet_min.loc["Iron, Fe"] = diet_min.loc["Iron, Fe"] / 1000.0

# Rename indices to match FoodNutrients rows.
diet_min.rename(index={"Fiber, total dietary": "Dietary Fiber",
                       "Carbohydrate, by difference": "Total Carbohydrate",
                       "Calcium, Ca": "Calcium",
                       "Iron, Fe": "Iron"}, inplace=True)

# -------------------------
# Process the maximum DRIs CSV (for Sodium)
# -------------------------
# Read the DRIs maximum file.
dri_max_df = pd.read_csv("diet_maximums.csv")
# Filter to obtain the row for sodium UL.
dri_max_df = dri_max_df[dri_max_df['Nutrition'].str.contains("Sodium")]
dri_max_df.set_index('Nutrition', inplace=True)
# Extract the value for the chosen group.
diet_max_value = dri_max_df.loc["Sodium, Na", group]
# Create a Series for diet_max with index "Sodium".
diet_max = pd.Series(diet_max_value, index=["Sodium"])
# Explicitly cast to float before dividing to avoid dtype issues.
diet_max["Sodium"] = diet_max["Sodium"].astype(float) / 1000.0

# -------------------------
# Solve the nutrition (subsistence) problem.
# -------------------------
tol = 1e-6
result = solve_subsistence_problem(FoodNutrients, Prices, diet_min, diet_max, tol=tol)

print("Cost of diet for {} is ${:4.2f} per day.\n".format(group, result.fun))
diet = result.diet

print("\nDiet (in 100s of grams):")
print(diet[diet >= tol])
print()

tab = pd.DataFrame({
    "Outcome": np.abs(result.A).dot(diet),
    "Recommendation": np.abs(result.b)
})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:, 1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())


Cost of diet for M 19-30 is $3.00 per day.


Diet (in 100s of grams):
name
Crunchy Peanut Butter Salted    3.057834
Shredded Mozzarella Cheese      1.057181
Jasmine Rice From Thailand      0.088509
Joe's O's Cereal                0.272160
Calrose Rice                    0.345628
dtype: float64


With the following nutritional outcomes of interest:
                        Outcome  Recommendation
Energy              2400.000000        2400.000
Protein              113.545243          56.000
Dietary Fiber         33.600000          33.600
Calcium                1.000000           1.000
Total Carbohydrate   130.000000         130.000
Iron                   0.008000           0.008
Sodium                 2.047830           2.300


Constraining nutrients are:
['Energy', 'Dietary Fiber', 'Calcium', 'Total Carbohydrate', 'Iron']


/tmp/ipykernel_477/4167779559.py:129: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.3' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  diet_max["Sodium"] = diet_max["Sodium"].astype(float) / 1000.0
